# <center>Table of Contents</center>

### 1. **Import Libraries**  
   - 1A. [Import Required Libraries](#1a-import-required-libraries)  
   - 1B. [Create Environment and Test](#1b-create-environment-and-test)  

### 2. **Train Model for Normal Version with PPO**  
   - 2A. [Preprocess Environment](#2a-preprocess-environment)  
   - 2B. [Train the Model](#2b-train-the-model)  
   - 2C. [Save the Model](#2c-save-the-model)  
   - 2D. [Evaluate the Model](#2d-evaluate-the-model)  

### 3. **Train Model for Hardcore Version with PPO**  
   - 3A. [Test the Environment](#3a-test-the-environment)  
   - 3B. [Preprocess Environment](#3b-preprocess-environment)  
   - 3C. [Train the Hardcore Model](#3c-train-the-hardcore-model)  
   - 3D. [Save the Hardcore Model](#3d-save-the-hardcore-model)  
   - 3E. [Evaluate the Hardcore Model](#3e-evaluate-the-hardcore-model)  

# <center>1. Import Libaries</center>

## 1A) Import Libaries

In [1]:
# Import the necessary libraries

# gymnasium is a modern version of the gym library, used to create and interact with reinforcement learning environments
import gymnasium as gym

# Import PPO (Proximal Policy Optimization) from stable-baselines3, which is a popular reinforcement learning algorithm
from stable_baselines3 import PPO, DQN, DDPG

# Import the evaluation function to assess the performance of the trained policy
from stable_baselines3.common.evaluation import evaluate_policy

# Import Monitor to log training information such as rewards and episode lengths
from stable_baselines3.common.monitor import Monitor

# Import utility functions for vectorized environments, normalization, frame stacking, and video recording
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecFrameStack, VecVideoRecorder

# Import os for handling directory creation and file paths
import os 

# Import pandas for handling and analyzing data (e.g., log files)
import pandas as pd

## 1B) Create Env and Test

In [ ]:
# Create the BipedalWalker environment with human-rendering mode enabled
env = gym.make("BipedalWalker-v3", render_mode="human")

In [ ]:
# Reset the environment (start a new episode) - without using seed or options
obs = env.reset()

# Let the agent take random actions for 1000 steps
for _ in range(1000):
    # Take a random action sampled from the environment's action space
    action = env.action_space.sample()
    
    # Step the environment forward using the chosen action
    # The environment returns the new observation (obs), the reward, 
    # whether the episode is done (done), if it was truncated (truncated), and additional info (info)
    obs, reward, done, truncated, info = env.step(action)
    
    # If the episode is finished (either done or truncated), reset the environment for a new episode
    if done or truncated:
        obs = env.reset()

# Close the environment when finished to clean up resources
env.close()

# <center>2) Train Model for Normal Version with PPO</center>

## 2A) Preprocces Enviorment

In [ ]:
env = gym.make("BipedalWalker-v3") #,render_mode = 'rgb_array') 

In [ ]:
# Define the logs directory and create it if it doesn't exist
logs_dir = 'logs'
os.makedirs(logs_dir, exist_ok=True)

# Specify the log filename (change this if needed)
log_filename = ""  # You can change this manually if needed. Default 'monitor.csv', if you add a text it wil be
                    #import as (text).monitor.csv

# Define the path for the monitor log
monitor_log_path = os.path.join(logs_dir, log_filename)

# Wrap the environment with Monitor and save logs to the defined path
env = Monitor(env, filename=monitor_log_path)

In [ ]:
# Wrap the environment in a DummyVecEnv to enable vectorized operations
env = DummyVecEnv([lambda: env])

# Normalize observations and rewards in the environment
# norm_obs: Normalize observations
# norm_reward: Normalize rewards
# clip_obs: Clip the observation values to prevent outliers
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# Stack the last n_stack observations (here n_stack=4) to provide temporal information to the agent
env = VecFrameStack(env, n_stack=4)

In [ ]:
video_folder = 'videos'
os.makedirs(video_folder, exist_ok=True)

env = VecVideoRecorder(env, video_folder, record_video_trigger=lambda x: x % 1000 == 0, video_length=200)

## 2B) Train Model

In [ ]:
# Create the PPO model with a Multi-Layer Perceptron (MLP) policy
model = PPO("MlpPolicy", env, verbose=1)

In [ ]:
model.learn(total_timesteps=1000000)

## 2C) Save Model

In [ ]:
model.save("ppo_bipedalwalker_1M")

In [ ]:
del model

## 2D) Evaluate Model

In [ ]:
model = PPO.load("ppo_bipedalwalker_1M")

In [ ]:
env = gym.make("BipedalWalker-v3", render_mode="human")

In [ ]:
# Evaluate the model (e.g., over 10 episodes)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Average reward: {mean_reward} ± {std_reward}")

Average Reward**: 248.39 ± 112.10
  - **Assessment**: This result indicates that the model is performing quite well overall. The average reward suggests that it has developed an effective policy and undergone a successful learning process. The high standard deviation (112.10) indicates that the model achieved significantly higher rewards in some trials while scoring lower in others, implying variability in its responses to different situations. This variability highlights the need for further analysis to understand how the model interacts with its environment.

# <center>3) Train Model for Hardcore Version with PPO</center>

## 3A) Test Enviroment

In [ ]:
env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="human")

In [ ]:
# Reset the environment (start a new episode) - without using seed or options
obs = env.reset()

# Let the agent take random actions for 1000 steps
for _ in range(1000):
    # Take a random action sampled from the environment's action space
    action = env.action_space.sample()
    
    # Step the environment forward using the chosen action
    # The environment returns the new observation (obs), the reward, 
    # whether the episode is done (done), if it was truncated (truncated), and additional info (info)
    obs, reward, done, truncated, info = env.step(action)
    
    # If the episode is finished (either done or truncated), reset the environment for a new episode
    if done or truncated:
        obs = env.reset()

# Close the environment when finished to clean up resources
env.close()

## 3B) Preprocces Enviorment

In [11]:
env = gym.make("BipedalWalker-v3", hardcore=True) #,render_mode = 'rgb_array')

In [12]:
# Define the logs directory and create it if it doesn't exist
logs_dir = 'logs'
os.makedirs(logs_dir, exist_ok=True)

# Specify the log filename (change this if needed)
log_filename = "5m_hardcore"  # You can change this manually if needed. Default 'monitor.csv', if you add a text it wil be
                    #import as (text).monitor.csv

# Define the path for the monitor log
monitor_log_path = os.path.join(logs_dir, log_filename)

# Wrap the environment with Monitor and save logs to the defined path
env = Monitor(env, filename=monitor_log_path)

In [13]:
# Wrap the environment in a DummyVecEnv to enable vectorized operations
env = DummyVecEnv([lambda: env])

# Normalize observations and rewards in the environment
# norm_obs: Normalize observations
# norm_reward: Normalize rewards
# clip_obs: Clip the observation values to prevent outliers
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# Stack the last n_stack observations (here n_stack=4) to provide temporal information to the agent
env = VecFrameStack(env, n_stack=4)

In [ ]:
# Define the video folder and create it if it doesn't exist
video_folder = 'videos'
os.makedirs(video_folder, exist_ok=True)

# Wrap the environment with VecVideoRecorder to record videos
# The recording is triggered every 1000 steps and each video will be 200 steps long
env = VecVideoRecorder(env, video_folder, record_video_trigger=lambda x: x % 1000 == 0, video_length=200)

## 3C) Train Model

In [14]:
# Create the PPO model with a Multi-Layer Perceptron (MLP) policy
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device


In [15]:
model.learn(total_timesteps=5000000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 818      |
|    ep_rew_mean     | -113     |
| time/              |          |
|    fps             | 3354     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.08e+03    |
|    ep_rew_mean          | -110        |
| time/                   |             |
|    fps                  | 2780        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011098673 |
|    clip_fraction        | 0.0916      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.67       |
|    explained_variance   | -0.0971     |
|    learning_rate        | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 703         |
|    ep_rew_mean          | -104        |
| time/                   |             |
|    fps                  | 2464        |
|    iterations           | 11          |
|    time_elapsed         | 9           |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.014261878 |
|    clip_fraction        | 0.147       |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.31       |
|    explained_variance   | 0.926       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00343     |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0347     |
|    std                  | 0.909       |
|    value_loss           | 0.0715      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 867         |
|    ep_rew_mean          | -104        |
| time/                   |             |
|    fps                  | 2443        |
|    iterations           | 20          |
|    time_elapsed         | 16          |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.016412206 |
|    clip_fraction        | 0.151       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.98       |
|    explained_variance   | 0.832       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0697     |
|    n_updates            | 190         |
|    policy_gradient_loss | -0.0389     |
|    std                  | 0.835       |
|    value_loss           | 0.0281      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 979         |
|    ep_rew_mean          | -103        |
| time/                   |             |
|    fps                  | 2443        |
|    iterations           | 29          |
|    time_elapsed         | 24          |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.021307435 |
|    clip_fraction        | 0.214       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.58       |
|    explained_variance   | 0.753       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0448     |
|    n_updates            | 280         |
|    policy_gradient_loss | -0.051      |
|    std                  | 0.756       |
|    value_loss           | 0.0182      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.01e+03    |
|    ep_rew_mean          | -101        |
| time/                   |             |
|    fps                  | 2448        |
|    iterations           | 38          |
|    time_elapsed         | 31          |
|    total_timesteps      | 77824       |
| train/                  |             |
|    approx_kl            | 0.023527618 |
|    clip_fraction        | 0.24        |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.16       |
|    explained_variance   | 0.812       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0826     |
|    n_updates            | 370         |
|    policy_gradient_loss | -0.0511     |
|    std                  | 0.681       |
|    value_loss           | 0.011       |
-----------------------------------------
----------------------------------------
| rollout/                |        

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.48e+03   |
|    ep_rew_mean          | -31        |
| time/                   |            |
|    fps                  | 1922       |
|    iterations           | 2399       |
|    time_elapsed         | 2555       |
|    total_timesteps      | 4913152    |
| train/                  |            |
|    approx_kl            | 0.33633065 |
|    clip_fraction        | 0.738      |
|    clip_range           | 0.2        |
|    entropy_loss         | 8.65       |
|    explained_variance   | 0.986      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0399    |
|    n_updates            | 23980      |
|    policy_gradient_loss | 0.0423     |
|    std                  | 0.028      |
|    value_loss           | 0.0011     |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.47e+03   |
|    ep_rew_mean          | -33.6      |
| time/                   |            |
|    fps                  | 1922       |
|    iterations           | 2409       |
|    time_elapsed         | 2565       |
|    total_timesteps      | 4933632    |
| train/                  |            |
|    approx_kl            | 0.62288404 |
|    clip_fraction        | 0.6        |
|    clip_range           | 0.2        |
|    entropy_loss         | 8.69       |
|    explained_variance   | 0.84       |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0485    |
|    n_updates            | 24080      |
|    policy_gradient_loss | -0.0246    |
|    std                  | 0.0273     |
|    value_loss           | 0.00956    |
----------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1.44e+03 |
|    ep_rew_mean          | -39.8    |
| time/                   |          |
|    fps                  | 1922     |
|    iterations           | 2419     |
|    time_elapsed         | 2576     |
|    total_timesteps      | 4954112  |
| train/                  |          |
|    approx_kl            | 10.57218 |
|    clip_fraction        | 0.872    |
|    clip_range           | 0.2      |
|    entropy_loss         | 8.81     |
|    explained_variance   | 0.979    |
|    learning_rate        | 0.0003   |
|    loss                 | -0.0541  |
|    n_updates            | 24180    |
|    policy_gradient_loss | -0.0151  |
|    std                  | 0.0264   |
|    value_loss           | 0.00453  |
--------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1.44e+03  |
|    ep_rew_mean      

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1.42e+03 |
|    ep_rew_mean          | -41.9    |
| time/                   |          |
|    fps                  | 1922     |
|    iterations           | 2429     |
|    time_elapsed         | 2587     |
|    total_timesteps      | 4974592  |
| train/                  |          |
|    approx_kl            | 2.275885 |
|    clip_fraction        | 0.769    |
|    clip_range           | 0.2      |
|    entropy_loss         | 8.89     |
|    explained_variance   | 0.988    |
|    learning_rate        | 0.0003   |
|    loss                 | -0.0242  |
|    n_updates            | 24280    |
|    policy_gradient_loss | -0.00612 |
|    std                  | 0.0261   |
|    value_loss           | 0.00348  |
--------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1.4e+03   |
|    ep_rew_mean      

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1.38e+03  |
|    ep_rew_mean          | -43.8     |
| time/                   |           |
|    fps                  | 1922      |
|    iterations           | 2439      |
|    time_elapsed         | 2597      |
|    total_timesteps      | 4995072   |
| train/                  |           |
|    approx_kl            | 0.4745683 |
|    clip_fraction        | 0.709     |
|    clip_range           | 0.2       |
|    entropy_loss         | 8.69      |
|    explained_variance   | 0.98      |
|    learning_rate        | 0.0003    |
|    loss                 | -0.0234   |
|    n_updates            | 24380     |
|    policy_gradient_loss | 0.00435   |
|    std                  | 0.0276    |
|    value_loss           | 0.00162   |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1.35e+03  |


## 3D) Save Model

In [16]:
model.save("ppo_bipedalwalker_hardcore_3M")

In [6]:
del model

## 3E) Evaluate Model 3M Model

In [7]:
model = PPO.load("models/ppo_bipedalwalker_hardcore_3M")

In [8]:
env = gym.make("BipedalWalker-v3", hardcore=True, render_mode="human")

In [ ]:
# Evaluate the model (e.g., over 10 episodes)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Average reward: {mean_reward} ± {std_reward}")

Average Reward**: -28.23 ± 24.82
  - **Assessment**: This result shows that the model is underperforming in the more challenging environment. A negative average reward indicates that the model mostly receives unfavorable feedback and struggles to achieve the target. The lower standard deviation (24.82) suggests less variability in performance, indicating that the model consistently performs poorly under difficult conditions. This may imply that the model requires more training and potentially different hyperparameter settings.

## 3F) Evaluate Model 5M Model

In [17]:
del model

In [18]:
model = PPO.load("models/ppo_bipedalwalker_hardcore_5M")

In [ ]:
# Evaluate the model (e.g., over 10 episodes)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1000)

print(f"Average reward: {mean_reward} ± {std_reward}")